In [1]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.engine import Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Reshape, Lambda, LSTM
from keras.layers import  K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import keras
import tensorflow
import tensorflow as tf

import time
import numpy as np

np.random.seed(1337)


class CustomImageDataGenerator(ImageDataGenerator):
    """
    Because Xception utilizes a custom preprocessing method, the only way to utilize this
    preprocessing method using the ImageDataGenerator is to overload the standardize method.

    The standardize method gets applied to each batch before ImageDataGenerator yields that batch.
    """

    def standardize(self, x):
        """
        Taken from keras.applications.xception.preprocess_input
        """
        if self.featurewise_center:
            x /= 255.
            x -= 0.5
            x *= 2.
        return x


def get_training_generator(batch_size=128):
    train_data_dir = '/home/divya/Documents/Minor_Project_Paper/CNN-RNN_implement/Potato/Train'
    validation_data_dir = '/home/divya/Documents/Minor_Project_Paper/CNN-RNN_implement/Potato/Valid'
    image_datagen = CustomImageDataGenerator(featurewise_center=True)

    train_generator = image_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size
    )

    val_generator = image_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=False
    )

    return train_generator, val_generator


def rgb_to_grayscale(input):
    """Average out each pixel across its 3 RGB layers resulting in a grayscale image"""
    return K.mean(input, axis=3)


def rgb_to_grayscale_output_shape(input_shape):
    return input_shape[:-1]


batch_size_phase_one = 32
batch_size_phase_two = 16
nb_val_samples = 5000

nb_epochs = 40

img_width = 299
img_height = 299

# Setting tensorbord callback
now = time.strftime("%c")
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir='./logs/' + 'cnn_rnn ' + now, histogram_freq=0, write_graph=True,
                                   write_images=False)

# Loading dataset
print("Loading the dataset with batch size of {}...".format(batch_size_phase_one))
train_generator, val_generator = get_training_generator(batch_size_phase_one)
print("Dataset loaded")

print("Building model...")
input_tensor = tf.keras.Input(shape=(img_width, img_height, 3))

# Creating CNN
cnn_model = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = tf.keras.layers.GlobalAveragePooling2D()(x)

# Make CNN layers not trainable
for layer in cnn_model.layers:
    layer.trainable = False

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = LSTM(2048, return_sequences=True)(x)
rnn_output = LSTM(2048)(x)

# Merging both cnn bottleneck and rnn's output wise element wise multiplication
x = tf.keras.layers.Concatenate(axis=-1)([cnn_bottleneck, rnn_output])
predictions = Dense(3, activation='softmax')(x)  

model = tf.keras.Model(inputs=input_tensor, outputs=predictions)

print("Model built")

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

print("Starting training")
checkpointer = ModelCheckpoint(filepath="./initial_cnn_rnn_weights_2.hdf5", verbose=1, save_best_only=True)

'''

fit_generator(
    generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None,
    validation_data=None, =None, validation_freq=1,
    class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False,
    shuffle=True, initial_epoch=0
)
'''


model.fit_generator(train_generator, steps_per_epoch=4480, epochs=nb_epochs, verbose=1,
                    validation_data=val_generator,
                    validation_steps=5000,
                    callbacks=[tensorboard_callback, checkpointer])

print("Initial training done, starting phase two (finetuning)")

# Load two new generator with smaller batch size, needed because using the same batch size
# for the fine tuning will result in GPU running out of memory and tensorflow raising an error
print("Loading the dataset with batch size of {}...".format(batch_size_phase_two))
train_generator, val_generator = get_training_generator(batch_size_phase_two)
print("Dataset loaded")


# Load best weights from initial training
model.load_weights("./initial_cnn_rnn_weights_2.hdf5")

# Make all layers trainable for finetuning
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

print("DKYL1")
checkpointer = ModelCheckpoint(filepath="./finetuned_cnn_rnn_weights_2.hdf5", verbose=1, save_best_only=True)
print("DKYL2")
model.fit_generator(train_generator, steps_per_epoch=2240, epochs=nb_epochs, verbose=1,
                    validation_data=val_generator,
                    validation_steps=5000,
                    callbacks=[tensorboard_callback, checkpointer])

# Final evaluation of the model
print("Training done, doing final evaluation...")

print("DKYL3")
model.load_weights("./finetuned_cnn_rnn_weights_2.hdf5")
print("DKYL4")
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

scores = model.evaluate_generator(val_generator, steps=nb_val_samples)
print(model.metrics_names, scores)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))



Using TensorFlow backend.


Loading the dataset with batch size of 32...
Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Dataset loaded
Building model...
Model built
Starting training
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
   1/4480 [..............................] - ETA: 0s - loss: 1.1074WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
  29/4480 [..............................] - ETA: 16:12:58 - loss: 1.2355WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 179200 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: val_loss improved fro